In [1]:
from models.snackpack import Snackpack
from models.schedule import Scheduler
import pandas as pd

## Problem 1) Load truck

In [2]:
df_products = pd.read_csv("data\Productos.csv", usecols=[0,1,2,3])

volumne_max = 180
weight_max = 40

In [3]:
modelo = Snackpack()
modelo.parameters(df_products, volumne_max, weight_max)
modelo.variables()
modelo.constrains()
modelo.objetive()
modelo.run_model()
modelo.print_result()

#############################################
[OPTIMAL] optimal solution profit 487.757153583 found
selected items: ['A', 'C', 'E', 'F', 'G', 'H', 'K', 'S', 'N']


['A', 'C', 'E', 'F', 'G', 'H', 'K', 'S', 'N']

## Problem 2) Scheduler Assignment

In [4]:
df_demand = pd.read_csv("data\scheduler - demand.csv")
df_availability = pd.read_csv("data\scheduler - availability.csv").set_index('Person')
collaborators = df_availability.index.unique().tolist()

In [5]:
def prep_data(df_demand):
    df_demand['fecha'] = pd.to_datetime(df_demand['fecha'])
    df_demand['Week'] = df_demand['fecha'].apply(lambda x: x.isocalendar().week)
    df_demand['Day'] = df_demand['fecha'].apply(lambda x: x.isocalendar().weekday - 1)
    week_list = df_demand['Week'].unique().tolist()
    df_demand = df_demand.drop(columns=['fecha']).set_index(['Week','Day'])
    periods = range(len(df_demand.columns))
    return (df_demand, week_list, periods)


In [6]:
availability_dict = {}
for i, row in df_availability.iterrows():
    availability_dict[i] = row.tolist()

In [7]:
df_demand, weeks, periods = prep_data(df_demand)
demand_dict_w = {}
demand_dict_d = {}
for i, row in df_demand.iterrows():
    demand_dict_d[i[1]] = row.tolist()
    demand_dict_w[i[0]] = demand_dict_d

In [8]:
modelo = Scheduler()
modelo.add_sets(weeks, periods)
modelo.parameters(collaborators, demand_dict_w, availability_dict)
modelo.variables()
modelo.constrains()
modelo.objetive()
modelo.run_model()

#############################################
[OPTIMAL] optimal solution profit 880.0 found


In [9]:
df_result = modelo.print_result()

Collaborador A week:6, day:0, periodo:0
Collaborador A week:6, day:1, periodo:0
Collaborador A week:6, day:2, periodo:0
Collaborador A week:6, day:3, periodo:2
Collaborador A week:6, day:4, periodo:0
Collaborador B week:6, day:1, periodo:0
Collaborador B week:6, day:3, periodo:0
Collaborador B week:6, day:4, periodo:1
Collaborador B week:6, day:5, periodo:0
Collaborador B week:6, day:6, periodo:1
Collaborador C week:6, day:0, periodo:2
Collaborador C week:6, day:1, periodo:1
Collaborador C week:6, day:3, periodo:1
Collaborador C week:6, day:5, periodo:1
Collaborador C week:6, day:6, periodo:1
Collaborador D week:6, day:0, periodo:2
Collaborador D week:6, day:2, periodo:2
Collaborador D week:6, day:3, periodo:2
Collaborador D week:6, day:4, periodo:2
Collaborador D week:6, day:5, periodo:2
Collaborador E week:6, day:0, periodo:1
Collaborador E week:6, day:3, periodo:1
Collaborador E week:6, day:4, periodo:1
Collaborador E week:6, day:5, periodo:1
Collaborador E week:6, day:6, periodo:1


In [10]:
demand_periods_list =[]
for i, row in df_demand.iterrows():
    demand_periods_list.extend(row.tolist())

In [11]:
df_result_shcedule = pd.pivot_table(df_result.reset_index(), index=['day','period'], aggfunc='count', values='Person')
schedule_periods_list = []
for i, row in df_result_shcedule.iterrows():
    schedule_periods_list.extend(row.tolist())

In [12]:
axis_x = []
for i in range(len(df_result_shcedule.index)):
    axis_x.append(str(df_result_shcedule.index.tolist()[i]))

In [13]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

trace1 = go.Bar(
    x=axis_x,
    y=demand_periods_list,
    name='Demanda'
)
trace2 = go.Bar(
    x=axis_x,
    y=schedule_periods_list,
    name='Scheduler'
)

fig = make_subplots(rows=1, cols=1, shared_xaxes=True)

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1,1)

In [14]:
fig

In [21]:
df_result.loc['D']

,day,period
week,,
6,0,2
6,2,2
6,3,2
6,4,2
6,5,2


In [16]:
df_result.query('day==5')

,,day,period
Person,week,,
B,6,5,0
C,6,5,1
D,6,5,2
E,6,5,1
F,6,5,0
G,6,5,0
H,6,5,1
I,6,5,1
J,6,5,2


In [ ]:
if 1 <= 5<= 1440:
    print('si')
else:
    print('no')

: 